In [352]:
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np

In [353]:
url = "http://www.boattrader.com/search-results/NewOrUsed-any/Type-any/Category-all/Zip-14853/Radius-200/Sort-Length:DESC"
main_url = "http://www.boattrader.com" #/listing/2006-sea-ray-320-sundancer-103069603/

def string_check(value):
    if value is None:
        return "" 
    return str(value.text)

def from_table(soup,name):
    
    data = list()
    
    if(soup.find('div',{'id':name}) is not None):
        table = soup.find('div',{'id':name}).find('table')
        rows = table.find_all('tr')
        for row in rows:
            if(name =='measurements'):
                index = string_check(row.find('dt'))
                val  = string_check(row.find('dd')) 
            else:
                index = string_check(row.find('th'))
                val  = string_check(row.find('td'))
            
            data.append(val)
            
    return data
        

def getsoup(url):
    r = urllib.urlopen(url).read()
    soup = BeautifulSoup(r,'html.parser')
    return soup

def check_element(element):
    return element if element is not None else " "


def scrape_page(s):
    
    a = s.find("h1",attrs={"class":"bd-name"})
    b = s.find("div",attrs={"class":"bd-price-location"})
    data = []
    if a is not None:
        year =  str(check_element(a.find('span',{"class":"bd-year"})).get_text())
        make =  str(check_element(a.find('span',{"class":"bd-make"})).get_text())
        model = str(check_element(a.find('span',{"class":"bd-model"})).get_text())
        data.append(year)
        data.append(make)
        data.append(model)
        
    if b is not None:                
        price = str(check_element(b.find('span',{"class":"bd-price contact-toggle"})).get_text()).split()[0]
        data.append(price)

    content = s.find('div',{'class':'detail-tbl'})

    
    
    if content is not None:
        data  = data + from_table(content,'boat-details')
        #data  = data + from_table(content,'measurements')
        #data  = data + from_table(content,'propulsion')
        #data  = data + from_table(content,'other-details')
                    
    print(data)
    return data

def scrape_full_page(url):
    soup = getsoup(url)
    a = soup.find_all("a", attrs={"data-reporting-click-listing-type":"standard listing"})
    links = [str(main_url+x['href']) for x in a]
    data = list()

    for link in links:
        s = getsoup(link)
        data.append(scrape_page(s))
    return data
    
    
#     details = s.find('section',{"id":"collapsible-content-areas"}) 
#     details = s.find('section',{"id":"collapsible-content-areas"}) 
#     from_table(details,'boat-details')


In [ ]:
df =[]
df = list(scrape_full_page(url))
soup = getsoup(url)
while soup.find('a',{'class':'next'}) is not None:
    url = str(main_url+link['href'])
    df.append(scrape_full_page(url))
    link = soup.find('a',{'class':'next'})
    url = str(main_url+link['href'])
    soup = getsoup(url)

columns = ['Year','Make','Model','Price','Class','Category','Year1','Make1','Length','Propulsion Type','Hull Material','Fuel Type','Location']
t = pd.DataFrame(df,columns = columns)


['2006', 'Sea Ray', '320 Sundancer', '$99,995', 'Power', ' Cruisers', '2006', 'Sea Ray', "320'", 'Other', 'Other', 'Other', 'N Tonawanda, NY']
['2012', 'Stingray Boats', '215LR', '$27,900', 'Power', ' Deck Boats', '2012', 'Stingray Boats', "263'", 'Other', 'Other', 'Other', 'Tafton, PA']
['2016', 'Yamaha Sport Boat', 'SX240', '$45,999', 'Power', ' Jet Boats', '2016', 'Yamaha Sport Boat', "240'", 'Other', 'Fiberglass', '', 'Staten Island, NY']
['1987', 'Blount', 'Luxury Dinner Yacht (GPC)', '$2,950,000', 'Power', ' Commercial Boats', '1987', 'Blount', "197'", 'Other', 'Steel', 'Diesel', 'Troy, NY']
['2016', 'Yamaha Sport Boat', '190 FSH DELUXE', '$26,999', 'Power', ' Jet Boats, Bowrider', '2016', 'Yamaha Sport Boat', "190'", 'Other', 'Fiberglass', 'Gas', 'Staten Island, NY']
['2016', 'Yamaha Sport Boat', '190 FSH SPORT', '$27,999', 'Power', ' Jet Boats', '2016', 'Yamaha Sport Boat', "190'", 'Other', 'Fiberglass', 'Gas', 'Staten Island, NY']
['2013', 'Aeroyacht', '125', '$28,303,680', 'S

['1998', 'Sea Ray', '630 Super Sun Sport', '$389,000', 'Power', ' Express Cruiser', '1998', 'Sea Ray', "63'", 'Twin Inboard', 'Fiberglass', 'Diesel', 'Stamford, CT']
['2018', 'Maritimo Yachts', 'M62', 'Request', 'Power', ' Motor Yachts, Flybridge', '2018', 'Maritimo Yachts', "62'", 'Other', 'Fiberglass', 'Diesel', 'Enroute To Staten Island, NY']
['1999', 'Offshore', '62 Pilot House', '$795,000', 'Power', ' Motor Yachts, Downeast', '1999', 'Offshore', "62'", 'Twin Inboard', 'Fiberglass', 'Diesel', 'Norwalk, CT']
['1989', 'Birchwood', '61 Motor Yacht TS/54', '$259,000', 'Power', ' Motor Yachts, Cruisers', '1989', 'Birchwood', "61'", 'Twin Inboard', 'Fiberglass', 'Diesel', 'City Island, NY']
['1981', 'Hatteras', '61 Motor Yacht', '$249,995', 'Power', ' Motor Yachts, Flybridge', '1981', 'Hatteras', "61'", 'Twin Inboard', 'Fiberglass', 'Diesel', 'Norwalk, CT']
['2001', 'Viking', 'Convertible', '$834,000', 'Power', ' Sports Fishing Boats', '2001', 'Viking', "61'", 'Twin Inboard', 'Fiberglass